In [0]:
from google.colab import drive
drive.mount('/content/drive')
# 必要ならば以下のようにディレクトリ移動する
%cd /content/drive/'My Drive'/'BatchEffectRemoval-master'/src

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/BatchEffectRemoval-master/src


In [0]:
!ls

Calibration_Util  main.ipynb	   __pycache__		Untitled1.ipynb
CostFunctions.py  Monitoring.py    ScatterHist.py	Untitled.ipynb
license.py	  OUH04_calib.csv  train_MMD_ResNet.py


In [0]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 47kB/s 
     |████████████████████████████████| 368kB 44.4MB/s 
     |████████████████████████████████| 3.2MB 42.4MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [0]:
import sys
sys.path.append('/content/drive/My Drive/BatchEffectRemoval-master/src/')
import CostFunctions as cf
import Monitoring as mn
import ScatterHist as sh

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [0]:
sys.path.append('/content/drive/My Drive/BatchEffectRemoval-master/src/Calibration_Util')
import DataHandler as dh 
import FileIO as io

In [0]:
'''
Created on Dec 5, 2016

@author: urishaham
'''

import os.path
import keras.optimizers
#from Calibration_Util import DataHandler as dh 
#from Calibration_Util import FileIO as io
from keras.layers import Input, Dense, merge, Activation, add
from keras.models import Model
from keras import callbacks as cb
import numpy as np
import matplotlib
from keras.layers.normalization import BatchNormalization
#detect display
import os
havedisplay = "DISPLAY" in os.environ
#if we have a display use a plotting backend
if havedisplay:
    matplotlib.use('TkAgg')
else:
    matplotlib.use('Agg')
    
#import CostFunctions as cf
#import Monitoring as mn
from keras.regularizers import l2
from sklearn import decomposition
from keras.callbacks import LearningRateScheduler
import math
#import ScatterHist as sh
from keras import initializers
from numpy import genfromtxt
import sklearn.preprocessing as prep
import tensorflow as tf
import keras.backend as K






# configuration hyper parameters
denoise = False # whether or not to train a denoising autoencoder to remove the zeros
keepProb=.8

# AE confiduration
ae_encodingDim = 25
l2_penalty_ae = 1e-2 

#MMD net configuration
mmdNetLayerSizes = [25, 25]
l2_penalty = 1e-2
#init = lambda shape, name:initializations.normal(shape, scale=.1e-4, name=name)
#def my_init (shape):
#    return initializers.normal(stddev=.1e-4)
#my_init = 'glorot_normal'

#######################
###### read data ######
#######################
# we load two CyTOF samples 

#data = 'person1_baseline'
data = 'OUH16_OUH05'

#if data =='person1_baseline':
#    sourcePath = os.path.join(io.DeepLearningRoot(),'Data/Person1Day1_baseline.csv')
#    targetPath = os.path.join(io.DeepLearningRoot(),'Data/Person1Day2_baseline.csv')
#if data =='person2_baseline':
#    sourcePath = os.path.join(io.DeepLearningRoot(),'Data/Person2Day1_baseline.csv')
#    targetPath = os.path.join(io.DeepLearningRoot(),'Data/Person2Day2_baseline.csv')
#if data =='person1_3month':
#    sourcePath = os.path.join(io.DeepLearningRoot(),'Data/Person1Day1_3month.csv')
#    targetPath = os.path.join(io.DeepLearningRoot(),'Data/Person1Day2_3month.csv')
#if data =='person2_3month':
#    sourcePath = os.path.join(io.DeepLearningRoot(),'Data/Person2Day1_3month.csv')
#    targetPath = os.path.join(io.DeepLearningRoot(),'Data/Person2Day2_3month.csv')

if data =='OUH04_OUH05':
    sourcePath = os.path.join(io.DeepLearningRoot(),'Data/OUH04_no_scale_imputed.csv')
    targetPath = os.path.join(io.DeepLearningRoot(),'Data/OUH05_no_scale_imputed.csv')
if data =='OUH10_OUH05':
    sourcePath = os.path.join(io.DeepLearningRoot(),'Data/OUH10_no_scale_imputed.csv')
    targetPath = os.path.join(io.DeepLearningRoot(),'Data/OUH05_no_scale_imputed.csv')
if data =='OUH16_OUH05':
    sourcePath = os.path.join(io.DeepLearningRoot(),'Data/OUH16_no_scale_imputed.csv')
    targetPath = os.path.join(io.DeepLearningRoot(),'Data/OUH05_no_scale_imputed.csv')

   
source = genfromtxt(sourcePath, delimiter=',', skip_header=0)
target = genfromtxt(targetPath, delimiter=',', skip_header=0)

# pre-process data: log transformation, a standard practice with CyTOF data
#target = dh.preProcessCytofData(target)
#source = dh.preProcessCytofData(source) 

numZerosOK=1
toKeepS = np.sum((source==0), axis = 1) <=numZerosOK
print(np.sum(toKeepS))
toKeepT = np.sum((target==0), axis = 1) <=numZerosOK
print(np.sum(toKeepT))

inputDim = target.shape[1]

if denoise:
    trainTarget_ae = np.concatenate([source[toKeepS], target[toKeepT]], axis=0)
    np.random.shuffle(trainTarget_ae)
    trainData_ae = trainTarget_ae * np.random.binomial(n=1, p=keepProb, size = trainTarget_ae.shape)
    input_cell = Input(shape=(inputDim,))
    encoded = Dense(ae_encodingDim, activation='relu',W_regularizer=l2(l2_penalty_ae))(input_cell)
    encoded1 = Dense(ae_encodingDim, activation='relu',W_regularizer=l2(l2_penalty_ae))(encoded)
    decoded = Dense(inputDim, activation='linear',W_regularizer=l2(l2_penalty_ae))(encoded1)
    autoencoder = Model(input=input_cell, output=decoded)
    autoencoder.compile(optimizer='rmsprop', loss='mse')
    autoencoder.fit(trainData_ae, trainTarget_ae, epochs=500, batch_size=128, shuffle=True,  validation_split=0.1,
                    callbacks=[mn.monitor(), cb.EarlyStopping(monitor='val_loss', patience=25,  mode='auto')])    
    source = autoencoder.predict(source)
    target = autoencoder.predict(target)

# rescale source to have zero mean and unit variance
# apply same transformation to the target
#preprocessor = prep.StandardScaler().fit(source)
#source = preprocessor.transform(source) 
#target = preprocessor.transform(target)    

#############################
######## train MMD net ######
#############################


calibInput = Input(shape=(inputDim,))
block1_bn1 = BatchNormalization()(calibInput)
block1_a1 = Activation('relu')(block1_bn1)
block1_w1 = Dense(mmdNetLayerSizes[0], activation='linear',kernel_regularizer=l2(l2_penalty), 
                  kernel_initializer=initializers.RandomNormal(stddev=1e-4))(block1_a1) 
block1_bn2 = BatchNormalization()(block1_w1)
block1_a2 = Activation('relu')(block1_bn2)
block1_w2 = Dense(inputDim, activation='linear',kernel_regularizer=l2(l2_penalty),
                  kernel_initializer=initializers.RandomNormal(stddev=1e-4))(block1_a2) 
block1_output = add([block1_w2, calibInput])
block2_bn1 = BatchNormalization()(block1_output)
block2_a1 = Activation('relu')(block2_bn1)
block2_w1 = Dense(mmdNetLayerSizes[1], activation='linear',kernel_regularizer=l2(l2_penalty), 
                  kernel_initializer=initializers.RandomNormal(stddev=1e-4))(block2_a1) 
block2_bn2 = BatchNormalization()(block2_w1)
block2_a2 = Activation('relu')(block2_bn2)
block2_w2 = Dense(inputDim, activation='linear',kernel_regularizer=l2(l2_penalty), 
                  kernel_initializer=initializers.RandomNormal(stddev=1e-4))(block2_a2) 
block2_output = add([block2_w2, block1_output])
block3_bn1 = BatchNormalization()(block2_output)
block3_a1 = Activation('relu')(block3_bn1)
block3_w1 = Dense(mmdNetLayerSizes[1], activation='linear',kernel_regularizer=l2(l2_penalty), 
                  kernel_initializer=initializers.RandomNormal(stddev=1e-4))(block3_a1) 
block3_bn2 = BatchNormalization()(block3_w1)
block3_a2 = Activation('relu')(block3_bn2)
block3_w2 = Dense(inputDim, activation='linear',kernel_regularizer=l2(l2_penalty), 
                  kernel_initializer=initializers.RandomNormal(stddev=1e-4))(block3_a2) 
block3_output = add([block3_w2, block2_output])

calibMMDNet = Model(inputs=calibInput, outputs=block3_output)

# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 150.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(step_decay)

#train MMD net
optimizer = keras.optimizers.rmsprop(lr=0.0)

calibMMDNet.compile(optimizer=optimizer, loss=lambda y_true,y_pred: 
               cf.MMD(block3_output,target,MMDTargetValidation_split=0.1).KerasCost(y_true,y_pred))
K.get_session().run(tf.global_variables_initializer())

sourceLabels = np.zeros(source.shape[0])
calibMMDNet.fit(source,sourceLabels,nb_epoch=500,batch_size=1000,validation_split=0.1,verbose=1,
           callbacks=[lrate, mn.monitorMMD(source, target, calibMMDNet.predict),
                      cb.EarlyStopping(monitor='val_loss',patience=50,mode='auto')])

##############################
###### evaluate results ######
##############################

calibratedSource = calibMMDNet.predict(source)

##################################### qualitative evaluation: PCA #####################################
pca = decomposition.PCA()
pca.fit(target)

# project data onto PCs
target_sample_pca = pca.transform(target)
projection_before = pca.transform(source)
projection_after = pca.transform(calibratedSource)

# choose PCs to plot
pc1 = 0
pc2 = 1
axis1 = 'PC'+str(pc1)
axis2 = 'PC'+str(pc2)
sh.scatterHist(target_sample_pca[:,pc1], target_sample_pca[:,pc2], projection_before[:,pc1], projection_before[:,pc2], axis1, axis2)
sh.scatterHist(target_sample_pca[:,pc1], target_sample_pca[:,pc2], projection_after[:,pc1], projection_after[:,pc2], axis1, axis2)
 
'''
# save models
autoencoder.save(os.path.join(io.DeepLearningRoot(),'savedModels/person1_baseline_DAE.h5'))                 
calibMMDNet.save_weights(os.path.join(io.DeepLearningRoot(),'savedModels/person1_baseline_ResNet_weights.h5'))  
'''


242756
242756
Instructions for updating:
Colocations handled automatically by placer.
setting scales using KNN
[2.2277150035916926, 4.455430007183385, 8.91086001436677]
setting all scale weights to 1
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:183: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 218480 samples, validate on 24276 samples
Epoch 1/500
218480/218480 [==============================] - 8s 36us/step - loss: 0.2560 - val_loss: 0.4213
Epoch 2/500
218480/218480 [==============================] - 6s 25us/step - loss: 0.1828 - val_loss: 0.4006
Epoch 3/500
218480/218480 [==============================] - 6s 25us/step - loss: 0.1756 - val_loss: 0.3890
Epoch 4/500
218480/218480 [==============================] - 6s 25us/step - loss: 0.1715 - val_loss: 0.3897
Epoch 5/500
218480/218480 [==============================] - 6s 25us/step - loss: 0.1690 - val_loss: 0.3885
Epoch 6/500
218480/218480 [==============================] - 6s 25us/step - loss: 0.1668 - val_loss: 0.3774
Epoch 7/500
218480/218480 [==============================] - 6s 26us/step - loss: 0.1647 - val_loss: 0.3796
Epoch 8/500
218480/218480 [==============================] - 6s 25us/step - loss: 0.1627 - val_loss: 0.3768
Epoch 9/500
218480/218480 [==============================] - 6s 26us/step - loss: 0.1

In [0]:
from google.colab import files
import csv

with open('/content/drive/My Drive/BatchEffectRemoval-master/Data/OUH16_no_scale_calib.csv', "w") as f:
    csv_w = csv.writer(f, lineterminator='\n')
    csv_w.writerows(calibratedSource)

#with open('/content/drive/My Drive/BatchEffectRemoval-master/Data/OUH05_calib.csv', "w") as f:
#    csv_w = csv.writer(f, lineterminator='\n')
#    csv_w.writerows(target)